In [4]:
import morphs
from ephys import core, rigid_pandas

In [2]:
block_path = morphs.paths.BLOCKS[0]

In [ ]:
# This is to clear the cache to make sure timing differences aren't due to OS cache paging
# This doesn't work on my notebook because I don't have sudo privilege, instead just run it in terminal
import os
os.system('sudo /sbin/sysctl -w vm.drop_caches=3')

In [3]:
%time morphs.data.load.ephys(block_path)

CPU times: user 10min 56s, sys: 5.23 s, total: 11min 1s
Wall time: 11min 8s


,cluster,recording,time_samples,stim_id,morph_dim,morph_pos,stim_presentation,stim_start,stim_duration,stim_aligned_time
0,30,0,23,ag064,ag,64,0,80067,0.39995,-4.00220
1,10,0,42,ag064,ag,64,0,80067,0.39995,-4.00125
2,13,0,98,ag064,ag,64,0,80067,0.39995,-3.99845
3,21,0,129,ag064,ag,64,0,80067,0.39995,-3.99690
4,22,0,129,ag064,ag,64,0,80067,0.39995,-3.99690
5,25,0,207,ag064,ag,64,0,80067,0.39995,-3.99300
6,2,0,271,ag064,ag,64,0,80067,0.39995,-3.98980
7,3,0,281,ag064,ag,64,0,80067,0.39995,-3.98930
8,16,0,305,ag064,ag,64,0,80067,0.39995,-3.98810
9,23,0,318,ag064,ag,64,0,80067,0.39995,-3.98745


In [5]:
def spikes_stims(block_path):
    spikes = core.load_spikes(block_path)

    stims = rigid_pandas.load_acute_stims(block_path)

    fs = core.load_fs(block_path)
    stims['stim_duration'] = stims['stim_end'] - stims['stim_start']
    rigid_pandas.timestamp2time(stims, fs, 'stim_duration')

    for rec, rec_group in stims.groupby('recording'):
        try:
            rec_group['stim_name'].astype(float)
            print('going to have to remove float stim recording ', rec)
            spikes = spikes[spikes['recording'] != rec]
            stims = stims[stims['recording'] != rec]
        except ValueError:
            if (rec_group['stim_duration'] > .41).any():
                print('removing long stim recording ', rec)
                spikes = spikes[spikes['recording'] != rec]
                stims = stims[stims['recording'] != rec]

    stim_ids = stims['stim_name']
    stim_ids = stim_ids.str.replace(r'_rec', '')
    stim_ids = stim_ids.str.replace(r'_rep\d\d', '')
    stims['stim_id'] = stim_ids
    morphs.data.parse.stim_id(stims)
    return spikes, stims, fs

In [ ]:
# This is to clear the cache to make sure timing differences aren't due to OS cache paging
os.system('sudo /sbin/sysctl -w vm.drop_caches=3')

In [6]:
%time spikes, stims, fs = spikes_stims(block_path)

CPU times: user 334 ms, sys: 380 ms, total: 714 ms
Wall time: 5.28 s


In [7]:
%%time
rigid_pandas.count_events(stims, index='stim_id')

spikes = spikes.join(rigid_pandas.align_events(spikes, stims,
                                               columns2copy=['stim_id', 'morph_dim', 'morph_pos',
                                                             'stim_presentation', 'stim_start', 'stim_duration']))

spikes['stim_aligned_time'] = (spikes['time_samples'].values.astype('int') -
                               spikes['stim_start'].values)
rigid_pandas.timestamp2time(spikes, fs, 'stim_aligned_time')

CPU times: user 11min 11s, sys: 4.53 s, total: 11min 16s
Wall time: 11min 15s


Looks like all the time is spent in the the alignment. Its not worth memoizing the loading of spikes and stims, and the alignment has to be done after any shuffling I do.